In [3]:
import os
import sys
import time
import warnings
from random import shuffle

import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from tensorflow.python.lib.io import file_io
from skimage.transform import resize
import numpy as np
import keras
from keras import backend as K
import cv2
from keras.models import Sequential
from scipy.stats import kurtosis, skew

warnings.filterwarnings("ignore")


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
# Window Length.
n = 100
STRIDE = 50

In [4]:
def getFeatures(data, fs):
    f1 = np.mean(data, axis=0)
    C = np.cov(data.T)
    f2 = np.concatenate((C[0, 0:3], C[1, 1:3], C[2, 2:3]))
    f3 = np.array([skew(data.iloc[:, 0]), skew(data.iloc[:, 1]), skew(data.iloc[:, 2])])
    f4 = np.array([kurtosis(data.iloc[:, 0]), kurtosis(data.iloc[:, 1]), kurtosis(data.iloc[:, 2])])
    f5 = np.zeros(3)
    f6 = np.zeros(3)
    for i in range(0,3):
        g = abs(np.fft.fft(data.iloc[:,i]))
        g = g[0:round(len(g)/2)]
        g[0] = 0
        max_i = np.argmax(g)
        f5[i] = g[max_i]
        f6[i] = fs * max_i
    return np.concatenate((f1, f2, f3, f4, f5, f6))

In [5]:
def generate_features(df, stride=STRIDE):
    df_out = pd.DataFrame(columns=[i for i in range(0, 42)])
    fs = 1 / n
    i = 0
    for i in range(0, df.shape[0] - n, stride):
        features_accel = getFeatures(df.iloc[i:i+n, 0:3], fs)
        features_gyro = getFeatures(df.iloc[i:i+n, 3:6], fs)
        features = np.concatenate((features_accel, features_gyro))
        df_out.loc[i] = features
        i += 1
    return df_out

In [6]:
# Choose via MODE ## TBD ##
def get_detection_window(df, stride=STRIDE):
    df_out = pd.DataFrame(columns=[0])
    i = 0
    for i in range(0, df.shape[0] - n, stride):
        df_slice = df.iloc[i:i+n,:]
        df_out.loc[i] = df_slice.mode().iloc[0,0]
#         df_out.loc[i] = df.iloc[i+n-1]
        i += 1
    return df_out

In [7]:
def extrapolate_predictions(predictions, size):
    extrapolated = np.fromiter((val for val in predictions for _ in range(0, STRIDE)), float)
    return extrapolated[0:size]

In [8]:
def read_test_data(session):
    df_data_arm = pd.read_csv('test_data/{}/armIMU.txt'.format(session), delim_whitespace=True, header=None)
    size = df_data_arm.shape[0]
    arm_features = generate_features(df_data_arm, stride=1)
    df_data_wrist = pd.read_csv('test_data/{}/wristIMU.txt'.format(session), delim_whitespace=True, header=None)
    wrist_features = generate_features(df_data_wrist, stride=1)
    zero_padding = pd.DataFrame(np.zeros(n))
    return size, pd.concat([arm_features, wrist_features], axis=1, sort=False, ignore_index=True)

In [7]:
from keras.layers import Conv2DTranspose, Dropout, Activation, BatchNormalization, UpSampling2D, Concatenate
from keras.layers import Input, Conv1D, MaxPooling1D, concatenate, TimeDistributed, Flatten, LSTM, Dense
from keras.models import Model

In [8]:
# Define model.
filters = 2
kernel_size = 2
model = Sequential()
# define CNN model
model.add(TimeDistributed(Conv1D(filters,kernel_size)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last')))
model.add(TimeDistributed(Flatten()))
# define LSTM model
model.add(LSTM(units=3, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, implementation=1, return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False))
model.add(Dense(units=3, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))

In [28]:
model.compile('adam', loss='mse', metrics=['mae', 'mape', 'acc'])

In [11]:
# Initializing dataframes for training and validation.
x_train = pd.DataFrame(columns=[i for i in range(0, 84)])
y_train = pd.DataFrame(columns=[0])
x_val = pd.DataFrame(columns=[i for i in range(0, 84)])
y_val = pd.DataFrame(columns=[0])

train_sessions = ['Session01', 'Session05', 'Session06', 'Session07', 'Session12']
# sessions = ['Session01_tmp']

for session in train_sessions:
    print('Processing training {}'.format(session))
    df_data_arm = pd.read_csv('TrainingData/{}/armIMU.txt'.format(session),  delim_whitespace=True, header=None)
    arm_features = generate_features(df_data_arm)
    df_data_wrist = pd.read_csv('TrainingData/{}/wristIMU.txt'.format(session), delim_whitespace=True, header=None)
    wrist_features = generate_features(df_data_wrist)
    features = pd.concat([arm_features, wrist_features], axis=1, sort=False, ignore_index=True)
    df_data_detection = pd.read_csv('TrainingData/{}/detection.txt'.format(session), header=None)
    x_train = x_train.append(other=features, ignore_index=True)
    y_train = y_train.append(other=get_detection_window(df_data_detection), ignore_index=True)

x_train.to_csv('x_train.txt', header=False, index=False, sep='\t', mode='a')
y_train.to_csv('y_train.txt', header=False, index=False, sep='\t', mode='a')
print('Saved training features!')
    
val_sessions = ['Session13']
for session in val_sessions:
    print('Processing validation {}'.format(session))
    df_data_arm = pd.read_csv('TrainingData/{}/armIMU.txt'.format(session),  delim_whitespace=True, header=None)
    arm_features = generate_features(df_data_arm)
    df_data_wrist = pd.read_csv('TrainingData/{}/wristIMU.txt'.format(session), delim_whitespace=True, header=None)
    wrist_features = generate_features(df_data_wrist)
    features = pd.concat([arm_features, wrist_features], axis=1, sort=False, ignore_index=True)
    df_data_detection = pd.read_csv('TrainingData/{}/detection.txt'.format(session), header=None)
    x_val = x_val.append(other=features, ignore_index=True)
    y_val = y_val.append(other=get_detection_window(df_data_detection), ignore_index=True)
    
x_val.to_csv('x_val.txt', header=False, index=False, sep='\t', mode='a')
y_val.to_csv('y_val.txt', header=False, index=False, sep='\t', mode='a')
print('Saved validation features!')

Processing training Session01
Processing training Session05
Processing training Session06
Processing training Session07
Processing training Session12
Saved training features!
Processing validation Session13
Saved validation features!


In [23]:
# x_train_df = pd.read_csv('x_train.txt', sep='\t')
# y_train_df = pd.read_csv('y_train.txt', sep='\t')
x_train_df = np.loadtxt('x_train.txt', delimiter='\t')
y_train_df = np.loadtxt('y_train.txt', delimiter='\t')

In [24]:
# x_val_df = pd.read_csv('x_val.txt', sep='\t')
# y_val_df = pd.read_csv('y_val.txt', sep='\t')
x_val_df = np.loadtxt('x_val.txt', delimiter='\t')
y_val_df = np.loadtxt('y_val.txt', delimiter='\t')

In [29]:
print(x_val_df.shape)
print(y_val_df.shape)

print(x_train_df.shape)
print(y_train_df.shape)

(371, 84)
(371,)
(18115, 84)
(18115,)


In [31]:
model.fit(x=x_train_df, y=y_train_df, epochs=5, verbose=1, callbacks=None, validation_split=0.0, validation_data=(x_val_df, y_val_df), shuffle=False, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_freq=1)




AttributeError: 'Sequential' object has no attribute '_feed_input_names'